# MNIST TRAINING
# Imports

In [1]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms

# NNI package for model quantization aware training (QAT).
from nni.algorithms.compression.pytorch.quantization import QAT_Quantizer
from nni.compression.pytorch.quantization.settings import set_quant_scheme_dtype

import sys
sys.path.append('../models')

# Import NaiveModel from naive.py
from naive_mnist import NaiveModel

c:\Users\ferga\anaconda3\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Training Functions

In [2]:
# Model training function
def train(model, device, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('{:2.0f}%  Loss {}'.format(100 * batch_idx / len(train_loader), loss.item()))


# Model testing function
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('Loss: {}  Accuracy: {}%)\n'.format(
        test_loss, 100 * correct / len(test_loader.dataset)))

## Train and Test

In [3]:
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Split dataset into train/test sets.
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, transform=trans),
    batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=trans),
    batch_size=1000, shuffle=True)


idim = next(iter(train_loader))[0][0].size()[1]
ifmap = next(iter(train_loader))[0][0].size()[0]

# Two things should be kept in mind when set this configure_list:
# 1. When deploying model on backend, some layers will be fused into one layer. For example, the consecutive
# conv + bn + relu layers will be fused into one big layer. If we want to execute the big layer in quantization
# mode, we should tell the backend the quantization information of the input, output, and the weight tensor of
# the big layer, which correspond to conv's input, conv's weight and relu's output.
# 2. Same tensor should be quantized only once. For example, if a tensor is the output of layer A and the input
# of the layer B, you should configure either {'quant_types': ['output'], 'op_names': ['a']} or
# {'quant_types': ['input'], 'op_names': ['b']} in the configure_list.

# Quantization configuration -
# conv1, conv2 layers have INT4 weight, INT4 input activations.
# relu layers have INT4 output activations
# FC layers have INT4 weight, input, output activations

# INT4 weight, INT4 activations
num_bits = 4

configure_list = [{
    'quant_types': ['weight', 'input'],
    'quant_bits': {'weight': num_bits, 'input': num_bits},
    'quant_start_step': 2,
    'op_names': ['conv1', 'conv2']
}, {
    'quant_types': ['output'],
    'quant_bits': {'output': num_bits},
    'quant_start_step': 2,
    'op_names': ['relu1', 'relu2', 'relu3']
}, {
    'quant_types': ['output', 'weight', 'input'],
    'quant_bits': {'output': num_bits, 'weight': num_bits, 'input': num_bits},
    'quant_start_step': 2,
    'op_names': ['fc1', 'fc2'],
}]

# you can also set the quantization dtype and scheme layer-wise through configure_list like:
# configure_list = [{
#         'quant_types': ['weight', 'input'],
#         'quant_bits': {'weight': 8, 'input': 8},
#         'op_names': ['conv1', 'conv2'],
#         'quant_dtype': 'int',
#         'quant_scheme': 'per_channel_symmetric'
#       }]
# For now quant_dtype's options are 'int' and 'uint. And quant_scheme's options are per_tensor_affine,
# per_tensor_symmetric, per_channel_affine and per_channel_symmetric.

# per_tensor_symmetric quantization scheme - see [Jacob et. al]
set_quant_scheme_dtype('weight', 'per_tensor_symmetric', 'int')
set_quant_scheme_dtype('output', 'per_tensor_symmetric', 'int')
set_quant_scheme_dtype('input', 'per_tensor_symmetric', 'int')

model = NaiveModel().to(device)
dummy_input = torch.randn(1, ifmap, idim, idim).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
# To enable batch normalization folding in the training process, you should
# pass dummy_input to the QAT_Quantizer.

quantizer = QAT_Quantizer(model, configure_list, optimizer, dummy_input=dummy_input)
quantizer.compress()

# Train and test/evaluate for 10 epoch
model.to(device)
for epoch in range(10):
    print('# Epoch {} #'.format(epoch))
    train(model, device, train_loader, optimizer)
    test(model, device, test_loader)

# Save quantized model parameters
model_path = "mnist_model.pth"
calibration_path = "mnist_calibration.pth"
onnx_path = "mnist_model.onnx"
input_shape = (1, ifmap, idim, idim)
torch.save(model.state_dict(), model_path)

# Quantize model with QAT_Quantizer (see NNI package)
#https://nni.readthedocs.io/en/stable/compression/quantization.html
# https://github.com/microsoft/nni

qmodel = NaiveModel().to(device)
dummy_input = torch.randn(1, ifmap, idim, idim).to(device)
optimizer = torch.optim.SGD(qmodel.parameters(), lr=0.01, momentum=0.5)
# To enable batch normalization folding in the training process, you should
# pass dummy_input to the QAT_Quantizer.
quantizer = QAT_Quantizer(qmodel, configure_list, optimizer, dummy_input=dummy_input)
quantizer.compress()
state = torch.load(model_path, map_location='cpu')
qmodel.load_state_dict(state, strict=True)
test(qmodel, device, test_loader)

# Epoch 0 #
 0%  Loss 2.3342342376708984
11%  Loss 1.9049136638641357
21%  Loss 1.457371711730957
32%  Loss 0.8283860087394714
43%  Loss 0.41125452518463135
53%  Loss 0.3375577926635742
64%  Loss 0.27770906686782837
75%  Loss 0.22761863470077515
85%  Loss 0.3718516528606415
96%  Loss 0.3804854154586792
Loss: 0.14416658401489257  Accuracy: 94.48%)

# Epoch 1 #
 0%  Loss 0.07342444360256195
11%  Loss 0.0912121906876564
21%  Loss 0.1208299845457077
32%  Loss 0.10878255218267441
43%  Loss 0.08095845580101013
53%  Loss 0.11199746280908585
64%  Loss 0.10481074452400208
75%  Loss 0.17088910937309265
85%  Loss 0.03318421170115471
96%  Loss 0.04575439170002937
Loss: 0.07975977210998535  Accuracy: 97.18%)

# Epoch 2 #
 0%  Loss 0.023166585713624954
11%  Loss 0.1449798196554184
21%  Loss 0.17774276435375214
32%  Loss 0.03368246182799339
43%  Loss 0.05983774736523628
53%  Loss 0.08120518922805786
64%  Loss 0.02431178092956543
75%  Loss 0.07956814765930176
85%  Loss 0.07321182638406754
96%  Loss 0.

'\'\'\nq_model = NaiveModel().to(device)\ndummy_input = torch.randn(1, ifmap, idim, idim).to(device)\noptimizer = torch.optim.SGD(q_model.parameters(), lr=0.01, momentum=0.5)\n    # To enable batch normalization folding in the training process, you should\n    # pass dummy_input to the QAT_Quantizer.\nquantizer = QAT_Quantizer(q_model, configure_list, optimizer, dummy_input=dummy_input)\nquantizer.compress()\n\nq_model.load_state_dict(torch.load("mnist_model.pth", map_location=device))\nq_model.eval()\n'